# 📌 <span style="font-size:18px; color:#007acc;"><b>Introduction</b></span>

This notebook is for **demonstration purposes only**. It focuses on data preprocessing and exploratory data analysis (EDA) for loan delinquency prediction. It starts with loading and cleaning the dataset, including handling missing values, detecting and removing outliers. Following this, we perform EDA, visualizing key numerical and categorical variables to better understand the data distributions and relationships. After feature engineering and normalization, the cleaned data is exported for model training and testing. **Note:** This work was conducted on **MS Azure**, so some settings may need adjustments. The dataset was obtained from **[Freddie Mac](https://www.freddiemac.com/)**. 

🚀 <span style="font-size:18px; color:#e63946;"><b>Let's get started!</b></span> 🚀


# <span style="font-size:18px; color:#007acc;"><b> Table of Contents</b></span>
1. 📊 [Import Libraries](#Import-Libraries) 
2. 📂 [Data Loading](#Data-Loading)  
3. 🧹 [Data Cleaning](#Data-Cleaning) 
4. 🛠️ [Feature Engineering](#Feature-Engineering) 
5. 📊 [Exploratory Data Analysis](#Exploratory-Data-Analysis) 
6. 🕵️‍♂️ [Detect Outliers with Scalable Unsupervised Outlier Detection (SUOD) framework](#Detect-Outliers-with-Scalable-Unsupervised-Outlier-Detection-(SUOD)-framework) 
7. 📏 [Feature Normalization](#Feature-normalization) 
8. 💾 [Export Data for Model Training and Testing](#Export-Data-for-Model-Training-and-Testing) 

## <span style="font-size:18px; color:#007acc;"><b> 1. Import Libraries <a id="Import-Libraries"></a> ##

In [ ]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pyod
import sklearn
import scipy
import joblib
import seaborn as sns
import category_encoders as ce
from suod.models.base import SUOD
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.hbos import HBOS
from pyod.models.ocsvm import OCSVM
from sklearn.decomposition import PCA
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer
from sklearn.preprocessing import OneHotEncoder
from pyod.models.iforest import IForest
from pyod.models import RandomForestRegressor
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

## <span style="font-size:18px; color:#007acc;"><b> 2. Data Loading <a id="Data-Loading"></a> ##

In [ ]:
############## CONNECT TO AZURE DATASTORE ##################
###########################################################
# Replace 'your_connection_string' with your actual Azure Blob Storage connection string
connect_str = 'your_connection_string_here'
# Instantiate a BlobServiceClient using a connection string
blob_service_client = BlobServiceClient.from_connection_string(connect_str)
# Replace with your actual container name
container_name = 'your_container_name_here'
container_client = blob_service_client.get_container_client(container_name)

############## IMPORT LOAN ORIGINATION DATA ################
###########################################################
# Define the year for data
year = '2004'
orifile = 'orig'
# Initialize an empty string to accumulate the content of each text file
combined_ori_04 = pd.DataFrame()
# Get a client to interact with the specified container
blobs_2004 = container_client.list_blobs()
for blob in blobs_2004:
    if keyword in blob.name:
         if orifile in blob.name:
            print(blob.name)
             blob_client = container_client.get_blob_client(blob)
             # Download and save the blob temporarily
             with open("temp_file.txt", "wb") as download_file:
                 download_file.write(blob_client.download_blob().readall())
             # Read the file into a DataFrame
             df = pd.read_csv("temp_file.txt", sep='|')  # Adjust delimiter as needed
             new_column_names_origination = ['Credit Score', 'First Payment Date', 'First Time Homebuyer Flag', 'Maturity Date', 'MSA', 'MI%', 'No of Units', 'Occupancy Status', 'CLTV', 'DTI Ratio', 'Ori UPB',
                                'Ori LTV', 'Ori Interest Rate', 'Channel', 'PPM Flag', 'Amortization Type', 'Property State', 'Property Type', 'Postal Code', 'Loan Seq Number', 'Loan Purpose',
                                'Ori Loan Term', 'Number of Borrowers', 'Seller Name', 'Servicer Name', 'Super Conforming Flag', 'Pre-relief Refinance Loan Seq Number', 'Program Indicator',
                                'Relief Refinance Indicator', 'Property Valuation Method', 'I/O Indicator', 'MI Cancellation Indicator'] 
             df.columns = new_column_names_origination # Original Column Names from Dataset
             Reordered_column_origination = ['Loan Seq Number', 'Credit Score', 'First Payment Date', 'First Time Homebuyer Flag', 'Maturity Date', 'MSA', 'MI%', 'No of Units', 'Occupancy Status', 'CLTV', 'DTI Ratio', 'Ori UPB',
                                'Ori LTV', 'Ori Interest Rate', 'Channel', 'PPM Flag', 'Amortization Type', 'Property State', 'Property Type', 'Postal Code', 'Loan Purpose',
                                'Ori Loan Term', 'Number of Borrowers', 'Seller Name', 'Servicer Name', 'Super Conforming Flag', 'Pre-relief Refinance Loan Seq Number', 'Program Indicator',
                                'Relief Refinance Indicator', 'Property Valuation Method', 'I/O Indicator', 'MI Cancellation Indicator']
             df = df[Reordered_column_origination]
             Reordered_to_keep_ori = ['Loan Seq Number', 'Credit Score', 'First Time Homebuyer Flag', 'MSA', 'No of Units', 'Occupancy Status', 'CLTV', 'DTI Ratio', 'Ori UPB',
                                'Ori Interest Rate', 'Channel', 'Amortization Type', 'Property State', 'Property Type', 'Postal Code', 'Loan Purpose',
                                'Ori Loan Term', 'Number of Borrowers', 'Servicer Name', 'I/O Indicator', 'MI Cancellation Indicator']
             df = df[Reordered_to_keep_ori]
             df = df.drop_duplicates(subset='Loan Seq Number', keep='last')
             combined_ori_04 = pd.concat([combined_ori_04, df])  # Appending content and a newline to separate files

############## IMPORT LOAN PERFORMANCE DATA ###############
###########################################################
# Define the year for data
year = '2004'
perffile = 'svcg'
# Initialize an empty string to accumulate the content of each text file
combined_perf_04 = pd.DataFrame()
# Get a client to interact with the specified container
blobs_2004 = container_client.list_blobs()
for blob in blobs_2004:
    if keyword in blob.name:
        if perffile in blob.name:
            print(blob.name)
            blob_client = container_client.get_blob_client(blob)
             # Download and save the blob temporarily
             with open("temp_file.txt", "wb") as download_file:
                 download_file.write(blob_client.download_blob().readall())
             # Read the file into a DataFrame
             df = pd.read_csv("temp_file.txt", sep='|')  # Adjust delimiter as needed
             new_column_names_performance = ['Loan Seq Number', 'Monthly reporting period','Current actual UPB','Current loan delinquency status','loan age','Remaining months to legal maturity',
                          'Defect settlement data','Modification flag','Zero balance code','ZB effective date','Current interest rate','current deferred UPB','DDLPI','MI recoveries',
                          'Net sale proceeds', 'Non MI recoveries', 'Total expenses','Legal costs','Maintenance and preservation costs','Taxes and insurance', 'Miscellaneous expenses',
                          'Actual loss calculation','Cumulative modification cost','Step modification flag','Payment deferral','ELTV','ZB removal UPB','Delinquent accrued interest','Delinquency due to disaster',
                          'Borrower assistance status code','Current month modification cost','Interest bearing UPB']
             df.columns = new_column_names_performance # Original Column Names from Dataset
             selected_columns_df_performance = ['Loan Seq Number', 'Current actual UPB', 'Current loan delinquency status', 'loan age', 'Remaining months to legal maturity','Current interest rate']
             df = df[selected_columns_df_performance]
             # Remove duplicates, keep the last occurrence
             df = df.drop_duplicates(subset='Loan Seq Number', keep='last')
             # Append to the combined DataFrame
             combined_perf_04 = pd.concat([combined_perf_04, df])

## MERGE ORIGINATION AND PERFORMANCE DATA ON 'LOAN SEQUENCE NUMBER' ##
# Merge data w.r.t this column
on_column = 'Loan Seq Number'
# Selected columns to keep from both DataFrames
selected_columns_df_performance = ['Loan Seq Number', 'Current actual UPB', 'Current loan delinquency status', 'loan age', 'Remaining months to legal maturity','Current interest rate']
selected_columns_df_performance = combined_perf_04[selected_columns_df_performance]
# Merge DataFrames on the specified column
merged_df = pd.merge(combined_ori_04, selected_columns_df_performance, on=on_column, how='inner')

## <span style="font-size:18px; color:#007acc;"><b> 3. Data Cleaning <a id="Data-Cleaning"></a> ##

In [ ]:
##################### CLEAN ORIGINATION DATA ##############
###########################################################
# Fill missing values for credit score with mean
valid_scores = merged_df[(merged_df['Credit Score'] >= 300) & (merged_df['Credit Score'] <= 900)]['Credit Score']
mean_score = valid_scores.mean()
merged_df['Credit Score'].fillna(mean_score, inplace=True)

# Fill missing values for FTHB flag with mode
mode_value = merged_df['First Time Homebuyer Flag'].mode()[0] 
merged_df['First Time Homebuyer Flag'].fillna(mode_value, inplace=True)

# Fill missing values with mode
merged_df['MSA'].fillna(0, inplace=True)
merged_df.loc[merged_df['MSA'] != 0, 'MSA'] = 1

# Fill missing values for number of units
mode_units = merged_df['No of Units'].mode()[0]
merged_df.loc[merged_df['No of Units'].isnull() | (merged_df['No of Units'] > 4), 'No of Units'] = mode_units

#Fill missing values for Occupancy status
#merged_df['Occupancy Status'].replace(9, np.nan, inplace=True)
mode_value = merged_df['Occupancy Status'].mode()[0]
merged_df['Occupancy Status'].fillna(mode_value, inplace=True)
#print(merged_df['Occupancy Status'])

#Fill missing values for CLTV
mean_score = merged_df['CLTV'].mean()
merged_df['CLTV'].fillna(mean_score, inplace=True)

#Fill missing values for DTI
valid_DTI = merged_df[(merged_df['DTI Ratio'] >= 0) & (merged_df['DTI Ratio'] <= 100)]['DTI Ratio']
mean_score = valid_DTI.mean()
thershold = 100
merged_df.loc[merged_df['DTI Ratio'] > thershold, 'DTI Ratio'] = np.nan
merged_df['DTI Ratio'].fillna(mean_score, inplace=True)

#Fill missing values for O_UPB
mean_score = merged_df['Ori UPB'].mean()
merged_df['Ori UPB'].fillna(mean_score, inplace=True)

#Fill missing values for Original Interest Rate
# For variable, check the bank rate, for fixed, take mean 
mean_score = merged_df['Ori Interest Rate'].mean()
merged_df['Ori Interest Rate'].fillna(mean_score, inplace=True)

#Fill missing values for Channel 
#merged_df['Channel'].replace(9, np.nan, inplace=True)
mode_value = merged_df['Channel'].mode()[0] 
merged_df['Channel'].fillna(mode_value, inplace=True)

#Fill missing values for Amortization type
mode_value = merged_df['Amortization Type'].mode()[0] 
merged_df['Amortization Type'].fillna(mode_value, inplace=True)

#Fill missing values for Property state
mode_value = merged_df['Property State'].mode()[0] 
merged_df['Property State'].fillna(mode_value, inplace=True)

#Fill missing values for Property type
merged_df['Property Type'].replace(99, np.nan, inplace=True)
mode_value = merged_df['Property Type'].mode()[0] 
merged_df['Property Type'].fillna(mode_value, inplace=True)

#Fill missing values for Loan Purpose
merged_df['Loan Purpose'].replace(9, np.nan, inplace=True)
mode_value = merged_df['Loan Purpose'].mode()[0] 
merged_df['Loan Purpose'].fillna(mode_value, inplace=True)

#Fill missing values for Original Loan Term
mode_value = merged_df['Ori Loan Term'].mode()[0] 
merged_df['Ori Loan Term'].fillna(mode_value, inplace=True)

#Fill missing values for Number of Borrowers
merged_df['Number of Borrowers'].replace(99, np.nan, inplace=True)
mode_value = merged_df['Number of Borrowers'].mode()[0] 
merged_df['Number of Borrowers'].fillna(mode_value, inplace=True)

#Fill missing values for Servicer Name
mode_value = merged_df['Servicer Name'].mode()[0] 
merged_df['Servicer Name'].fillna(mode_value, inplace=True)

#Fill missing values for I/O Indicator
mode_value = merged_df['I/O Indicator'].mode()[0] 
merged_df['I/O Indicator'].fillna(mode_value, inplace=True)

#Fill missing values for MI Cancellation Indicator
merged_df['MI Cancellation Indicator'].replace('7', np.nan, inplace=True)
mode_value = merged_df['MI Cancellation Indicator'].mode()[0] 
merged_df['MI Cancellation Indicator'].fillna(mode_value, inplace=True)
#print(merged_df['MI Cancellation Indicator'])

############## CLEAN PERFORMANCE DATA #################
#######################################################

#Fill missing values for Current Actual UPB
mode_value = merged_df['Current actual UPB'].mode()[0] 
merged_df['Current actual UPB'].fillna(mode_value, inplace=True)

#Fill missing values for Current Loan Deliquency Status
value_to_assign = 4 
for index, value in merged_df['Current loan delinquency status'].items():
    if str(value).isalpha():
        merged_df.at[index, 'Current loan delinquency status'] = value_to_assign
mode_value = merged_df['Current loan delinquency status'].mode()[0] 
merged_df['Current loan delinquency status'].fillna(mode_value, inplace=True)

#Fill missing values for Loan Age
mean_value = merged_df['loan age'].mean() 
merged_df['loan age'].fillna(mean_value, inplace=True)

#Fill missing values for Remaining Months for Maturity
mean_value = merged_df['Remaining months to legal maturity'].mean()
merged_df['Remaining months to legal maturity'].fillna(mean_value, inplace=True)

#Fill missing values for Current Interest Rate
mean_value = merged_df['Current interest rate'].mode()[0] 
merged_df['Current interest rate'].fillna(mean_value, inplace=True)

## CHECKING FOR MISSING VALUES ##
print("\nMissing Values:")
print(merged_df.isnull().sum())
plt.figure(figsize=(12, 8))
sns.heatmap(merged_df.isnull(), cbar=False, cmap='viridis')
plt.title('Missing Data Heatmap')
plt.show()


## <span style="font-size:18px; color:#007acc;"><b>4. Feature Engineering <a id="Feature-Engineering"></a> ##

In [ ]:
############## First Time Homebuyer Flag ##################
###########################################################
# Initialize OneHotEncoder
one_hot_encoder = OneHotEncoder()
encoded_data = one_hot_encoder.fit_transform(merged_df[['First Time Homebuyer Flag']])
#Convert the result to a DataFrame
df_encoded = pd.DataFrame(encoded_data.toarray(), columns=one_hot_encoder.get_feature_names_out(['First Time Homebuyer Flag']))
# Drop the original column
merged_df.drop(columns=['First Time Homebuyer Flag'], inplace=True)
# Concatenate the encoded columns with the original DataFrame
merged_df = pd.concat([merged_df, df_encoded], axis=1)
if 'First Time Homebuyer Flag_N' in merged_df.columns: #This column is dropped to avoid feature's linear dependency
    merged_df.drop(columns=['First Time Homebuyer Flag_N'], inplace=True)
new_column_name = {'First Time Homebuyer Flag_Y': 'First Time Homebuyer Flag'}
merged_df.rename(columns=new_column_name, inplace=True)

############## Occupancy Status ###########################
###########################################################
# Initialize OneHotEncoder
one_hot_encoder = OneHotEncoder()
encoded_data = one_hot_encoder.fit_transform(merged_df[['Occupancy Status']])
#Convert the result to a DataFrame
df_encoded = pd.DataFrame(encoded_data.toarray(), columns=one_hot_encoder.get_feature_names_out(['Occupancy Status']))
# Drop the original column
merged_df.drop(columns=['Occupancy Status'], inplace=True)
# Concatenate the encoded columns with the original DataFrame
merged_df = pd.concat([merged_df, df_encoded], axis=1)
merged_df.drop(columns=['Occupancy Status_S'], inplace=True)
new_column_name = {'Occupancy Status_I': 'Occupancy Investor'}
merged_df.rename(columns=new_column_name, inplace=True)
new_column_name = {'Occupancy Status_P': 'Occupancy Principal'}
merged_df.rename(columns=new_column_name, inplace=True)

############## Amortization Type ##########################
###########################################################
# Initialize OneHotEncoder
one_hot_encoder = OneHotEncoder()
encoded_data = one_hot_encoder.fit_transform(merged_df[['Amortization Type']])
#Convert the result to a DataFrame
df_encoded = pd.DataFrame(encoded_data.toarray(), columns=one_hot_encoder.get_feature_names_out(['Amortization Type']))
# Drop the original column
merged_df.drop(columns=['Amortization Type'], inplace=True, axis = ['Columns'])
# Concatenate the encoded columns with the original DataFrame
merged_df = pd.concat([merged_df, df_encoded], axis=1)
if 'Amortization Type_ARM' in merged_df.columns:
    merged_df.drop(columns=['Amortization Type_ARM'], inplace=True)
new_column_name = {'Amortization Type_FRM': 'Amortization Type'}
merged_df.rename(columns=new_column_name, inplace=True)

############## Property State ############################
###########################################################
# Initialize OneHotEncoder
one_hot_encoder = OneHotEncoder()
encoded_data = one_hot_encoder.fit_transform(merged_df[['Property State']])
#Convert the result to a DataFrame
df_encoded = pd.DataFrame(encoded_data.toarray(), columns=one_hot_encoder.get_feature_names_out(['Property State']))
# Drop the original column
merged_df.drop(columns=['Property State'], inplace=True)
# Concatenate the encoded columns with the original DataFrame
merged_df = pd.concat([merged_df, df_encoded], axis=1)
if 'Property State_AK' in merged_df.columns:
    merged_df.drop(columns=['Property State_AK'], inplace=True)

############## Property Type ##################
###########################################################
# Initialize OneHotEncoder
one_hot_encoder = OneHotEncoder()
encoded_data = one_hot_encoder.fit_transform(merged_df[['Property Type']])
#Convert the result to a DataFrame
df_encoded = pd.DataFrame(encoded_data.toarray(), columns=one_hot_encoder.get_feature_names_out(['Property Type']))
# Drop the original column
merged_df.drop(columns=['Property Type'], inplace=True)
# Concatenate the encoded columns with the original DataFrame
merged_df = pd.concat([merged_df, df_encoded], axis=1)
if 'Property Type_SF' in merged_df.columns:
    merged_df.drop(columns=['Property Type_SF'], inplace=True)

############## Loan Purpose ##################
###########################################################
# Initialize OneHotEncoder
one_hot_encoder = OneHotEncoder()
encoded_data = one_hot_encoder.fit_transform(merged_df[['Loan Purpose']])
#Convert the result to a DataFrame
df_encoded = pd.DataFrame(encoded_data.toarray(), columns=one_hot_encoder.get_feature_names_out(['Loan Purpose']))
# Drop the original column
merged_df.drop(columns=['Loan Purpose'], inplace=True)
# Concatenate the encoded columns with the original DataFrame
merged_df = pd.concat([merged_df, df_encoded], axis=1)
if 'Loan Purpose_R' in merged_df.columns:
    merged_df.drop(columns=['Loan Purpose_R'], inplace=True)
new_column_name = {'Loan Purpose_P': 'Purchase'}
merged_df.rename(columns=new_column_name, inplace=True)
new_column_name = {'Loan Purpose_C': 'Cash'}
merged_df.rename(columns=new_column_name, inplace=True)
new_column_name = {'Loan Purpose_N': 'No Cash Out'}
merged_df.rename(columns=new_column_name, inplace=True)

############## Servicer Name ##################
###########################################################
# Initialize OneHotEncoder
one_hot_encoder = OneHotEncoder()
encoded_data = one_hot_encoder.fit_transform(merged_df[['Servicer Name']])
#Convert the result to a DataFrame
df_encoded = pd.DataFrame(encoded_data.toarray(), columns=one_hot_encoder.get_feature_names_out(['Servicer Name']))
# Drop the original column
merged_df.drop(columns=['Servicer Name'], inplace=True)
# Concatenate the encoded columns with the original DataFrame
merged_df = pd.concat([merged_df, df_encoded], axis=1)
if 'Servicer Name_Other servicers' in merged_df.columns:
    merged_df.drop(columns=['Servicer Name_Other servicers'], inplace=True)

############## Channel ##################
###########################################################
# Initialize OneHotEncoder
one_hot_encoder = OneHotEncoder()
encoded_data = one_hot_encoder.fit_transform(merged_df[['Channel']])
#Convert the result to a DataFrame
df_encoded = pd.DataFrame(encoded_data.toarray(), columns=one_hot_encoder.get_feature_names_out(['Channel']))
# Drop the original column
merged_df.drop(columns=['Channel'], inplace=True)
# Concatenate the encoded columns with the original DataFrame
merged_df = pd.concat([merged_df, df_encoded], axis=1)
if 'Channel_TBO' in merged_df.columns:
    merged_df.drop(columns=['Channel_TBO'], inplace=True)

############## I/O Indicator ##################
###########################################################
# Initialize OneHotEncoder
one_hot_encoder = OneHotEncoder()
encoded_data = one_hot_encoder.fit_transform(merged_df[['I/O Indicator']])
#Convert the result to a DataFrame
df_encoded = pd.DataFrame(encoded_data.toarray(), columns=one_hot_encoder.get_feature_names_out(['I/O Indicator']))
# Drop the original column
merged_df.drop(columns=['I/O Indicator'], inplace=True)
# Concatenate the encoded columns with the original DataFrame
merged_df = pd.concat([merged_df, df_encoded], axis=1)
if 'I/O Indicator_Y' in merged_df.columns:
    merged_df.drop(columns=['I/O Indicator_Y'], inplace=True)
new_column_name = {'I/O Indicator_N': 'I/O Indicator'}
merged_df.rename(columns=new_column_name, inplace=True)

############## MI Cancellation Indicator ##################
###########################################################
# Initialize OneHotEncoder
one_hot_encoder = OneHotEncoder()
encoded_data = one_hot_encoder.fit_transform(merged_df[['MI Cancellation Indicator']])
#Convert the result to a DataFrame
df_encoded = pd.DataFrame(encoded_data.toarray(), columns=one_hot_encoder.get_feature_names_out(['MI Cancellation Indicator']))
# Drop the original column
merged_df.drop(columns=['MI Cancellation Indicator'], inplace=True)
# Concatenate the encoded columns with the original DataFrame
merged_df = pd.concat([merged_df, df_encoded], axis=1)
if 'MI Cancellation Indicator_N' in merged_df.columns:
    merged_df.drop(columns=['MI Cancellation Indicator_N'], inplace=True)
new_column_name = {'MI Cancellation Indicator_Y': 'MI Cancellation Indicator_Y'}
merged_df.rename(columns=new_column_name, inplace=True)
print(merged_df.shape)

## <span style="font-size:18px; color:#007acc;"><b> 5. Exploratory Data Analysis <a id="Exploratory-Data-Analysis"></a> ##

In [ ]:
# Set plot style
sns.set(style="whitegrid")

# 1. Check basic information about the dataset
print("Dataset Info:")
print(merged_df.info())

# 2. Summary statistics of numerical columns
print("\nSummary Statistics:")
print(merged_df.describe())

# 5. Distribution of Credit Score (and other numerical columns)
plt.figure(figsize=(12, 6))
sns.histplot(merged_df['Credit Score'], kde=True, color='skyblue', bins=50)
plt.title('Distribution of Credit Score')
plt.xlabel('Credit Score')
plt.ylabel('Frequency')
plt.show()

# 6. Distribution of Loan Amount (Ori UPB)
plt.figure(figsize=(12, 6))
sns.histplot(merged_df['Ori UPB'], kde=True, color='salmon', bins=50)
plt.title('Distribution of Original Loan Amount (Ori UPB)')
plt.xlabel('Original Loan Amount')
plt.ylabel('Frequency')
plt.show()

# 7. Count plot for categorical columns (e.g., First Time Homebuyer Flag)
plt.figure(figsize=(12, 6))
sns.countplot(data=merged_df, x='First Time Homebuyer Flag', palette='Set2')
plt.title('First Time Homebuyer Flag Distribution')
plt.xlabel('First Time Homebuyer Flag')
plt.ylabel('Count')
plt.show()

# 8. Correlation heatmap (for numerical columns)
plt.figure(figsize=(12, 8))
corr = merged_df.corr()  # Make sure to use only numerical columns
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()

# 9. Pair plot to visualize relationships between numerical variables
sns.pairplot(merged_df[['Credit Score', 'Ori UPB', 'DTI Ratio', 'CLTV', 'loan age']])
plt.suptitle('Pairplot of Numerical Variables', y=1.02)
plt.show()

# 10. Boxplot for potential outliers in numerical columns
plt.figure(figsize=(12, 6))
sns.boxplot(data=merged_df[['Credit Score', 'Ori UPB', 'DTI Ratio', 'CLTV', 'loan age']])
plt.title('Boxplot of Numerical Variables')
plt.show()

# 11. Checking distribution of Loan Delinquency Status
plt.figure(figsize=(12, 6))
sns.countplot(data=merged_df, x='Current loan delinquency status', palette='Set1')
plt.title('Loan Delinquency Status Distribution')
plt.xlabel('Loan Delinquency Status')
plt.ylabel('Count')
plt.show()

# 12. Visualize relationship between Credit Score and DTI Ratio
plt.figure(figsize=(12, 6))
sns.scatterplot(data=merged_df, x='Credit Score', y='DTI Ratio', alpha=0.6)
plt.title('Credit Score vs DTI Ratio')
plt.xlabel('Credit Score')
plt.ylabel('DTI Ratio')
plt.show()

# 13. Visualize relationship between Ori UPB and CLTV
plt.figure(figsize=(12, 6))
sns.scatterplot(data=merged_df, x='Ori UPB', y='CLTV', alpha=0.6)
plt.title('Original Loan Amount vs CLTV')
plt.xlabel('Original Loan Amount')
plt.ylabel('CLTV')
plt.show()

# 14. Checking for duplicates and dropping them if necessary
print("\nDuplicate Rows:")
print(merged_df.duplicated().sum())

# 15. Explore categorical variables relationship with numerical variables
plt.figure(figsize=(12, 6))
sns.boxplot(data=merged_df, x='Loan Purpose', y='Credit Score')
plt.title('Credit Score by Loan Purpose')
plt.xticks(rotation=45)
plt.show()

## <span style="font-size:18px; color:#007acc;"><b> 6. Detect Outliers with Scalable Unsupervised Outlier Detection (SUOD) framework <a id="Detect-Outliers-with-Scalable-Unsupervised-Outlier-Detection-(SUOD)-framework"></a> ##

In [ ]:
######## DETECT OUTLIERS WITH SUOD ########
###########################################
# Numerical Data
anomaly_inputs = ['Credit Score', 'CLTV', 'No of Units', 'DTI Ratio', 'Ori UPB', 'Ori Interest Rate', 'Ori Loan Term', 
                  'Number of Borrowers', 'Current actual UPB', 'Current loan delinquency status', 'loan age', 
                  'Remaining months to legal maturity', 'Current interest rate']

X_train = merged_df[anomaly_inputs]
print("step1")
# Set up your base estimators
contamination = 0.01  # 1% of the data are outliers
base_estimators = [#IForest(n_estimators=200),
    #IForest(n_estimators=100)
    #LOF(n_neighbors=5, contamination=contamination),
    #LOF(n_neighbors=15, contamination=contamination),
    #LOF(n_neighbors=25, contamination=contamination),
    #HBOS(contamination=contamination),
    #OCSVM(contamination=contamination),
    #KNN(n_neighbors=5, contamination=contamination),
    #KNN(n_neighbors=15, contamination=contamination),
    KNN(n_neighbors=25, contamination=contamination)
    ]  
# Initialize SUOD model
model = SUOD(base_estimators=base_estimators, n_jobs=1,
             rp_flag_global=True, bps_flag=True,
             approx_flag_global=True, contamination=contamination)
# Fit the model
model.fit(X_train)
model.approximate(X_train)  # conduct model approximation if it is enabled
# Predict labels for each base estimator
predicted_labels = model.predict(X_train)
# Calculate the average of predicted labels across all estimators
average_labels = np.mean(predicted_labels, axis=1)
# Add the average labels to X_train
X_train_with_avg_labels = np.hstack((X_train, average_labels.reshape(-1, 1)))
# Separate outliers and inliers based on the average labels
outliers_avg = X_train_with_avg_labels[average_labels >= 0.01]
inliers_avg = X_train_with_avg_labels[average_labels < 0.01]
def convert_to_float(value):
    try:
        return float(value)
    except ValueError:
        try:
            return float(value.replace("'", ""))  # Handle string representations of floats
        except ValueError:
            return np.nan  # Handle non-convertible values, you can modify this as needed
inliers_avg = np.vectorize(convert_to_float)(inliers_avg)
outliers_avg= np.vectorize(convert_to_float)(outliers_avg)
print(outliers_avg.shape)
print(inliers_avg.shape)

# Funciton to plot outliers
def plot_outliers_inliers(outliers, inliers, x_col, y_col, x_label, y_label, title):
    plt.figure(figsize=(8, 6))
    # Plot outliers
    plt.scatter(outliers[:, x_col], outliers[:, y_col], color='red', label='Outliers (Avg)')
    # Plot inliers
    plt.scatter(inliers[:, x_col], inliers[:, y_col], color='green', label='Inliers (Avg)')
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.legend()
    plt.grid(True)
    plt.show()

# Define column indices for each plot
columns = [(0, 1), (2, 3), (4, 5), (6, 7), (8, 9), (10, 11)]
labels = [
    ('Credit', 'CLTV'),
    ('No of Units', 'DTI_Ratio'),
    ('Ori UPB', 'Ori Interest Rate'),
    ('Ori Loan Term', 'Number of Borrowers'),
    ('Current actual UPB', 'Current loan delinquency status'),
    ('Loan age', 'Remaining months to legal maturity')
]
titles = [
    'Outliers and Inliers (Average) - Credit vs CLTV',
    'Outliers and Inliers (Average) - No of Units vs DTI_Ratio',
    'Outliers and Inliers (Average) - Ori UPB vs Ori Interest Rate',
    'Outliers and Inliers (Average) - Ori Loan Term vs Number of Borrowers',
    'Outliers and Inliers (Average) - Current actual UPB vs Current loan delinquency status',
    'Outliers and Inliers (Average) - Loan age vs Remaining months to legal maturity'
]

# Loop through each set of column indices and plot
for (x_col, y_col), (labels_pair, title) in zip(columns, zip(labels, titles)):
    plot_outliers_inliers(outliers_avg, inliers_avg, x_col, y_col, labels_pair[0], labels_pair[1], title)

######## ADD OUTLIER COLUMN TO DATASET ########
###############################################
df_OD = pd.DataFrame(average_labels)
merged_df_OD = pd.concat([merged_df, df_OD], axis=1)
df_merged_OD = pd.DataFrame(merged_df_OD)

# Check for null values
null_values = df_merged_OD.isnull().values.any()
# Check for NaN values
nan_values = df_merged_OD.isna().values.any()
if null_values or nan_values:
    print("There are null or NaN values in the DataFrame.")
else:
    print("There are no null or NaN values in the DataFrame.")
df_merged_OD.shape

######## DELETE ROWS BASED ON OUTLIERS ########
###############################################
# Remove outliers from the original dataset
df_merged_OD_WO_Outliers = df_merged_OD[df_merged_OD.iloc[:, -1] < 0.5]
df_merged_OD_WO_Outliers.head()
print(df_merged_OD_WO_Outliers.shape)
unique_values = df_merged_OD_WO_Outliers['Current loan delinquency status'].unique()
print(f"Unique values in column {'Current loan delinquency status'}: {unique_values}")

######## REPLACE CURRENT LOAN DELIQUENCY STATUS > 1 with 1 ########
####################################################################
# Clip values greater than 1 in the 'Current loan delinquency status' column
df_merged_OD_WO_Outliers['Current loan delinquency status'] = df_merged_OD_WO_Outliers['Current loan delinquency status'].clip(upper=1)

# Count occurrences of each value in the 'Current loan delinquency status' column
status_counts = df_merged_OD_WO_Outliers['Current loan delinquency status'].value_counts()

# Print counts for each status directly
status_counts.apply(lambda count, status: print(f"Number of {status}s in column 'B': {count}"), 
                    status=status_counts.index, 
                    count=status_counts.values)

# Get the position of 'Current loan delinquency status' column
position = df_merged_OD_WO_Outliers.columns.get_loc('Current loan delinquency status')
print(f"Position of 'Current loan delinquency status' column: {position}")

# Shape of the final dataframe
print(f"Shape of the final dataframe: {df_merged_OD_WO_Outliers.shape}")


## <span style="font-size:18px; color:#007acc;"><b>7. Feature Normalization <a id="Feature-normalization"></a> ##

In [ ]:
# Columns to be normalized
columns_to_normalize = ['Credit Score', 'CLTV', 'No of Units', 'DTI Ratio', 'Ori UPB', 'Ori Interest Rate', 'Ori Loan Term', 
                  'Number of Borrowers', 'Current actual UPB', 'Current loan delinquency status', 'loan age', 
                  'Remaining months to legal maturity', 'Current interest rate']

# Initialize StandardScaler
normalizer = MinMaxScaler()

# Normalize selected columns
df_merged_OD_WO_Outliers[columns_to_normalize] = normalizer.fit_transform(df_merged_OD_WO_Outliers[columns_to_normalize])
unique_values = df_merged_OD_WO_Outliers['loan age'].unique()
print(f"Unique values in column {'loan age'}: {unique_values}")

## <span style="font-size:18px; color:#007acc;"><b>8. Export Data for Model Training and Testing <a id="Export-Data-for-Model-Training-and-Testing"></a> ##

In [ ]:
##### DROP OUTLIER LABEL COLUMN #####
#####################################
df_merged_OD_WO_Outliers = df_merged_OD_WO_Outliers.iloc[:, :-1]
# Drop the 'Loan Seq Number' column (serial number column)
df_merged_OD_WO_Outliers.drop(columns='Loan Seq Number', inplace=True)
# Drop the 'Postal Code' column
df_merged_OD_WO_Outliers.drop(columns='Postal Code', inplace=True)
# Print the shape of the final cleaned dataframe
print(f"Shape of cleaned dataframe: {df_merged_OD_WO_Outliers.shape}")
output_df = df_merged_OD_WO_Outliers[['Current loan delinquency status']]
output_df['Current loan delinquency status'] = output_df['Current loan delinquency status'].replace({0: 1, 1: 0})
# Drop the 'Current loan delinquency status' column from the main dataframe
df_merged_OD_WO_Outliers.drop(columns=['Current loan delinquency status'], inplace=True)

###### WRITE DATA TO EXCEL FILES ######
######################################
input_output_files = [
    ('feature_file_0405.xlsx', df_merged_OD_WO_Outliers),  # Features file
    ('label_file_0405.xlsx', output_df)  # Labels file
]
# Write each DataFrame to a separate Excel file
for file_name, data_frame in input_output_files:
    data_frame.to_excel(file_name, index=False)
print(f"Feature and label files have been saved successfully.")